In [1]:
import os
import urllib

import numpy as np
import tensorflow as tf
import pandas as pd


IRIS_TRAINING = "../data/iris/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "../data/iris/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

BATCH_SIZE=10




/Users/tfox/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
   
    train = pd.read_csv(IRIS_TRAINING, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(IRIS_TEST, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y, test_x, test_y)

train_x, train_y, test_x, test_y = load_data()

In [3]:
# Specify that all features have real-value data
feature_columns = []
for key in train_x.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [4]:

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb2dea3940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
# Define the input function

def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

# Fit model.
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE), steps=1000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 15.610294, step = 1
INFO:tensorflow:global_step/sec: 578.851
INFO:tensorflow:loss = 4.180154, step = 101 (0.174 sec)
INFO:tensorflow:global_step/sec: 904.313
INFO:tensorflow:loss = 1.0358124, step = 201 (0.111 sec)
INFO:tensorflow:global_step/sec: 787.602
INFO:tensorflow:loss = 0.20880494, step = 301 (0.127 sec)
INFO:tensorflow:global_step/sec: 828.498
INFO:tensorflow:loss = 0.058251597, step = 401 (0.120 sec)
INFO:tensorflow:global_step/sec: 956.464
INFO:tensorflow:loss = 0.31146997, step = 501 (0.105 sec)
INFO:tensorflow:global_step/sec: 965.213
INFO:tensorflow:loss = 0.08877989, step = 601 (0.104 sec)
INFO:tensorflow:global_step/sec: 1028.72
INFO:tensorfl

In [6]:

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    
    features=dict(features)
    inputs = features if labels is None else (features, labels)
   
    # Convert the inputs to a Dataset.
    return tf.data.Dataset.from_tensor_slices(inputs).batch(batch_size)


# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-22-04:49:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-22-04:49:09
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.065629, global_step = 1000, loss = 0.65629
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/iris_model/model.ckpt-1000

Test set accuracy: 0.967



In [7]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x, test_y[:3], batch_size=BATCH_SIZE))

list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([  1.1274962,  -6.0248094, -13.965336 ], dtype=float32),
  'probabilities': array([9.9921727e-01, 7.8244368e-04, 2.7856464e-07], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-6.6753   ,  1.2756677, -4.7903156], dtype=float32),
  'probabilities': array([3.5138187e-04, 9.9733436e-01, 2.3142896e-03], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-11.39478   ,  -0.42954504,   3.3908927 ], dtype=float32),
  'probabilities': array([3.7089200e-07, 2.1448089e-02, 9.7855157e-01], dtype=float32),
  'class_ids': array([2]),
  'classes': array([b'2'], dtype=object)}]